In [100]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from math import radians

import matplotlib.pyplot as plt


import prep_plays

Acquire.py Loaded Successfully
Prep.py Loaded Successfully
Prep.py Loaded Successfully


Can we figure out the distance between the nearest defender and the intended reciever?
- Steps to go about solving this problem
    - merging play.csv to week{number}.csv
        - play id is only unique to game id, can concat both numbers to create a unique number
    - using play description to find the intended receiver
        - will be the second name in the list of names
        - given as first initial and last name
    - extract the name
    - match the name given the play id
        - the week{number}.csv gives both first and last name
    - out of all players involved in the play, who is the closest defender

# Prep week csv

## clean nfl weeks(drop noise)

In [2]:
def filter_nfl_weeks():
    
    for i in range(1,18):
        #read in csv
        df = pd.read_csv('week' + str(i) + '.csv')
        #keep only pass_forward events
        df = df[(df.event == 'pass_forward')]
        #fill in empty positions with football
        df.position = df.position.fillna('BALL')
        #reset index
        df.reset_index(drop=True)
        #save as new csv
        df.to_csv('week' + str(i) + 'filtered.csv', index=False)
        #print week number when done
        print(f'{i}')

In [ ]:
filter_nfl_weeks()

## plays

In [101]:
#read in csv
plays = prep_plays.prep_plays_for_weeks()

In [102]:
plays.T

,0,1,2,3,4,5,6,7,8,9,...,16997,16998,16999,17000,17001,17002,17003,17004,17005,17006
gameId,2018090600,2018090600,2018090600,2018090600,2018090600,2018090600,2018090600,2018090600,2018090600,2018090600,...,2018122201,2018122201,2018122201,2018122201,2018122201,2018122201,2018122201,2018122201,2018122201,2018122201
playId,75,146,168,190,256,320,344,402,492,521,...,3471,3493,3518,3745,3817,3842,3884,3909,3931,3953
playDescription,(15:00) M.Ryan pass short right to J.Jones pus...,(13:10) M.Ryan pass incomplete short right to ...,(13:05) (Shotgun) M.Ryan pass incomplete short...,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,(10:59) (Shotgun) M.Ryan pass incomplete short...,(10:10) (Shotgun) N.Foles pass short left to N...,(9:24) (Shotgun) N.Foles pass incomplete short...,(9:08) M.Ryan pass incomplete deep left to M.S...,(7:01) M.Ryan pass short left to T.Coleman pus...,(6:19) M.Ryan pass short left to A.Hooper to P...,...,(7:39) (Shotgun) P.Rivers pass incomplete shor...,(7:35) (Shotgun) P.Rivers pass short right to ...,(6:51) P.Rivers pass short right to J.Jackson ...,(2:54) (Shotgun) P.Rivers pass short left to A...,(2:40) (Shotgun) P.Rivers pass short left to K...,(2:19) (Shotgun) P.Rivers pass short right to ...,(2:00) (Shotgun) P.Rivers pass short left to K...,(1:41) (Shotgun) P.Rivers pass incomplete shor...,(1:37) (Shotgun) P.Rivers pass incomplete deep...,(1:31) (Shotgun) P.Rivers pass deep middle int...
quarter,1,1,1,1,1,1,1,1,1,1,...,4,4,4,4,4,4,4,4,4,4
down,1,1,2,3,3,2,3,1,2,1,...,2,3,1,1,1,2,1,1,2,3
yardsToGo,15,10,10,10,1,8,4,10,13,10,...,6,6,10,20,10,2,10,10,10,10
team_by_comp_yds,3,3,3,3,3,8,8,3,3,3,...,19,19,19,19,19,19,19,19,19,19
defendersInTheBox,7,7,6,6,8,7,6,7,8,8,...,6,6,6,6,6,6,4,5,4,6
numberOfPassRushers,4,4,4,5,6,4,4,4,4,3,...,6,5,4,5,5,4,5,5,4,4
QB_under_pressure,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
#since playId is only unique to game id we need to combine to make a unique identifier
#first have to change the data types to strings
plays.gameId = plays.gameId.astype(str)
plays.playId = plays.playId.astype(str)

In [104]:
#concat to create a unique identifier
plays['playid'] = plays.gameId + plays.playId

In [105]:
#drop old columns
plays = plays.drop(columns = {'gameId', 'playId'})

In [131]:
plays.playDescription[140]

'(:23) (Shotgun) T.Taylor pass deep right intended for J.Gordon INTERCEPTED by C.Sutton at PIT 8. C.Sutton to PIT 8 for no gain (J.Gordon).'

In [125]:
plays.head(40)

,playDescription,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,...,JUMBO,PISTOL,SHOTGUN,SINGLEBACK,WILDCAT,four_three,three_four,nickel,dime,playid
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,0,0,0,0,0,0,0,1,0,201809060075
1,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,3,7.0,4.0,0.0,13:10:00,49.0,...,0,0,0,1,0,0,0,1,0,2018090600146
2,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,3,6.0,4.0,0.0,13:05:00,49.0,...,0,0,1,0,0,0,0,1,0,2018090600168
3,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,3,6.0,5.0,1.0,13:01:00,49.0,...,0,0,1,0,0,0,0,0,1,2018090600190
4,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,3,8.0,6.0,0.0,10:59:00,11.0,...,0,0,1,0,0,0,0,0,0,2018090600256
5,(10:10) (Shotgun) N.Foles pass short left to N...,1,2,8,8,7.0,4.0,0.0,10:10:00,14.0,...,0,0,1,0,0,0,0,1,0,2018090600320
6,(9:24) (Shotgun) N.Foles pass incomplete short...,1,3,4,8,6.0,4.0,0.0,09:24:00,18.0,...,0,0,0,0,0,0,0,1,0,2018090600344
7,(9:08) M.Ryan pass incomplete deep left to M.S...,1,1,10,3,7.0,4.0,1.0,09:08:00,54.0,...,0,0,0,1,0,0,0,1,0,2018090600402
8,(7:01) M.Ryan pass short left to T.Coleman pus...,1,2,13,3,8.0,4.0,0.0,07:01:00,46.0,...,0,0,0,1,0,0,0,1,0,2018090600492
9,(6:19) M.Ryan pass short left to A.Hooper to P...,1,1,10,3,8.0,3.0,0.0,06:19:00,20.0,...,0,0,0,1,0,0,0,1,0,2018090600521


## week1

In [108]:
#read in csv
wk1 = pd.read_csv('week1filtered.csv')

In [109]:
wk1.T

,0,1,2,3,4,5,6,7,8,9,...,14156,14157,14158,14159,14160,14161,14162,14163,14164,14165
time,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,2018-09-07T01:07:18.099Z,...,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z,2018-09-11T05:29:36.000Z
x,96.43,84.82,80.69,86.02,82.65,69.63,93.81,86.1,79.51,76.53,...,22.53,46.81,38.55,38.82,26.66,35.19,35.85,40.85,28.48,47.63
y,26.78,38.33,44.91,18.64,34.56,30.81,27.3,33.53,20,44.93,...,19.63,29.69,23.2,15.91,7.72,25.58,38.2,46.18,36.19,29.64
s,0.84,5.53,3.11,3.52,6.35,2.93,3.92,3.75,2.99,4.68,...,1.59,0.97,3.56,4.27,2.64,2.14,2.68,0.92,0.93,2.16
a,2.05,4.3,6.67,0.69,1.21,2.65,3.23,1.49,0.69,2.18,...,0.98,0.74,1.92,1.92,0.83,1.38,1.33,4.07,2.26,3.9
dis,0.08,0.54,0.34,0.35,0.64,0.3,0.39,0.37,0.3,0.47,...,0.16,0.1,0.35,0.43,0.26,0.22,0.26,0.08,0.09,0.23
o,356.4,295.34,155.11,28.19,262.06,79.24,293.03,52.23,78.25,182.14,...,49.01,8.39,316.47,272.12,49.1,91.21,79.6,156.73,79.63,NaN
dir,333.39,318.58,297.88,216.15,272.78,315.98,332.35,311.29,259.75,277.48,...,309.79,51.72,311.26,277.16,282.71,333.38,343.6,134.27,27.26,NaN
event,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,...,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward,pass_forward
nflId,310,79848,2.49545e+06,2.49561e+06,2.53304e+06,2.53483e+06,2.54358e+06,2.55232e+06,2.55269e+06,2.55538e+06,...,2.54349e+06,2.5435e+06,2.5506e+06,2.55249e+06,2.55249e+06,2.55628e+06,2.55659e+06,2.55688e+06,2.55818e+06,NaN


In [110]:
#since playId is only unique to game id we need to combine to make a unique identifier
#first have to change the data types to strings
wk1.gameId = wk1.gameId.astype(str)

wk1.playId = wk1.playId.astype(str)

In [111]:
#concat to create a unique identifier
wk1['playid'] = wk1.gameId + wk1.playId

In [112]:
#drop old columns
wk1 = wk1.drop(columns = {'gameId', 'playId'})

In [113]:
wk1

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route,playid
0,2018-09-07T01:07:18.099Z,96.43,26.78,0.84,2.05,0.08,356.40,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,left,NaN,201809060075
1,2018-09-07T01:07:18.099Z,84.82,38.33,5.53,4.30,0.54,295.34,318.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN,201809060075
2,2018-09-07T01:07:18.099Z,80.69,44.91,3.11,6.67,0.34,155.11,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH,201809060075
3,2018-09-07T01:07:18.099Z,86.02,18.64,3.52,0.69,0.35,28.19,216.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN,201809060075
4,2018-09-07T01:07:18.099Z,82.65,34.56,6.35,1.21,0.64,262.06,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH,201809060075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14161,2018-09-11T05:29:36.000Z,35.19,25.58,2.14,1.38,0.22,91.21,333.38,pass_forward,2556277.0,Marqui Christian,41.0,SS,26,away,left,NaN,20180910014192
14162,2018-09-11T05:29:36.000Z,35.85,38.20,2.68,1.33,0.26,79.60,343.60,pass_forward,2556593.0,Cory Littleton,58.0,ILB,26,away,left,NaN,20180910014192
14163,2018-09-11T05:29:36.000Z,40.85,46.18,0.92,4.07,0.08,156.73,134.27,pass_forward,2556885.0,Jalen Richard,30.0,RB,26,home,left,SCREEN,20180910014192
14164,2018-09-11T05:29:36.000Z,28.48,36.19,0.93,2.26,0.09,79.63,27.26,pass_forward,2558183.0,John Johnson,43.0,SS,26,away,left,NaN,20180910014192


In [114]:
#testing unique identifier
wk1[wk1.playid == '201809060075']

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route,playid
0,2018-09-07T01:07:18.099Z,96.43,26.78,0.84,2.05,0.08,356.40,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,left,NaN,201809060075
1,2018-09-07T01:07:18.099Z,84.82,38.33,5.53,4.30,0.54,295.34,318.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN,201809060075
2,2018-09-07T01:07:18.099Z,80.69,44.91,3.11,6.67,0.34,155.11,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH,201809060075
3,2018-09-07T01:07:18.099Z,86.02,18.64,3.52,0.69,0.35,28.19,216.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN,201809060075
4,2018-09-07T01:07:18.099Z,82.65,34.56,6.35,1.21,0.64,262.06,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH,201809060075
5,2018-09-07T01:07:18.099Z,69.63,30.81,2.93,2.65,0.30,79.24,315.98,pass_forward,2534832.0,Rodney McLeod,23.0,FS,36,home,left,NaN,201809060075
6,2018-09-07T01:07:18.099Z,93.81,27.30,3.92,3.23,0.39,293.03,332.35,pass_forward,2543583.0,Devonta Freeman,24.0,RB,36,away,left,NaN,201809060075
7,2018-09-07T01:07:18.099Z,86.10,33.53,3.75,1.49,0.37,52.23,311.29,pass_forward,2552315.0,Jordan Hicks,58.0,MLB,36,home,left,NaN,201809060075
8,2018-09-07T01:07:18.099Z,79.51,20.00,2.99,0.69,0.30,78.25,259.75,pass_forward,2552689.0,Ronald Darby,21.0,CB,36,home,left,NaN,201809060075
9,2018-09-07T01:07:18.099Z,76.53,44.93,4.68,2.18,0.47,182.14,277.48,pass_forward,2555383.0,Jalen Mills,31.0,CB,36,home,left,NaN,201809060075


## merge

In [204]:
#merge 
df = pd.merge(plays, wk1, left_on = 'playid', right_on = 'playid', how = 'inner')

In [205]:
df.drop_duplicates(inplace=True)

In [206]:
#testing to see if new result matches old result
df[df.playid == '201809060075']

,playDescription,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,...,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,left,NaN
1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,318.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN
2,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH
3,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,216.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN
4,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH
5,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,315.98,pass_forward,2534832.0,Rodney McLeod,23.0,FS,36,home,left,NaN
6,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,332.35,pass_forward,2543583.0,Devonta Freeman,24.0,RB,36,away,left,NaN
7,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,311.29,pass_forward,2552315.0,Jordan Hicks,58.0,MLB,36,home,left,NaN
8,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,259.75,pass_forward,2552689.0,Ronald Darby,21.0,CB,36,home,left,NaN
9,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,277.48,pass_forward,2555383.0,Jalen Mills,31.0,CB,36,home,left,NaN


In [207]:
df

,playDescription,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,...,dir,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,333.39,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,left,NaN
1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,318.58,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN
2,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,297.88,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH
3,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,216.15,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN
4,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,272.78,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13423,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,333.38,pass_forward,2556277.0,Marqui Christian,41.0,SS,26,away,left,NaN
13424,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,343.60,pass_forward,2556593.0,Cory Littleton,58.0,ILB,26,away,left,NaN
13425,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,134.27,pass_forward,2556885.0,Jalen Richard,30.0,RB,26,home,left,SCREEN
13426,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,27.26,pass_forward,2558183.0,John Johnson,43.0,SS,26,away,left,NaN


## Identify reciever

In [145]:
import re

In [208]:
#extracting names from play description
#second name will be the intended reciever
desc = df.playDescription.str.findall(r'(\b[A-Z]+\.\b[A-Z]+\w+)').apply(','.join)

In [209]:
desc

0            M.Ryan,J.Jones,M.Jenkins
1            M.Ryan,J.Jones,M.Jenkins
2            M.Ryan,J.Jones,M.Jenkins
3            M.Ryan,J.Jones,M.Jenkins
4            M.Ryan,J.Jones,M.Jenkins
                     ...             
13423    D.Carr,J.Richard,M.Christian
13424    D.Carr,J.Richard,M.Christian
13425    D.Carr,J.Richard,M.Christian
13426    D.Carr,J.Richard,M.Christian
13427    D.Carr,J.Richard,M.Christian
Name: playDescription, Length: 13428, dtype: object

In [210]:
#split the desc namees by comma
temp = desc.str.split(',', expand = True)

In [211]:
temp

,0,1,2,3,4,5,6,7,8,9
0,M.Ryan,J.Jones,M.Jenkins,None,None,None,None,None,None,None
1,M.Ryan,J.Jones,M.Jenkins,None,None,None,None,None,None,None
2,M.Ryan,J.Jones,M.Jenkins,None,None,None,None,None,None,None
3,M.Ryan,J.Jones,M.Jenkins,None,None,None,None,None,None,None
4,M.Ryan,J.Jones,M.Jenkins,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
13423,D.Carr,J.Richard,M.Christian,None,None,None,None,None,None,None
13424,D.Carr,J.Richard,M.Christian,None,None,None,None,None,None,None
13425,D.Carr,J.Richard,M.Christian,None,None,None,None,None,None,None
13426,D.Carr,J.Richard,M.Christian,None,None,None,None,None,None,None


In [203]:
#checking reciever column for nulls
#there are some
#will have to write a loop that considers nulls and skips them
temp[1].isnull().value_counts()

False    13263
True       165
Name: 1, dtype: int64

In [212]:
# obtaining last name from WR column
# originally first inital dot last name
WR = temp[1].str.split('.', expand = True)

In [214]:
#saving last name of receiver
df['reciever_last_name'] = WR[1]

In [215]:
#checking dataframe
df

,playDescription,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,...,event,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route,reciever_last_name
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,pass_forward,310.0,Matt Ryan,2.0,QB,36,away,left,NaN,Jones
1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,pass_forward,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN,Jones
2,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,pass_forward,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH,Jones
3,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,pass_forward,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN,Jones
4,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,pass_forward,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH,Jones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13423,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,pass_forward,2556277.0,Marqui Christian,41.0,SS,26,away,left,NaN,Richard
13424,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,pass_forward,2556593.0,Cory Littleton,58.0,ILB,26,away,left,NaN,Richard
13425,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,pass_forward,2556885.0,Jalen Richard,30.0,RB,26,home,left,SCREEN,Richard
13426,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,pass_forward,2558183.0,John Johnson,43.0,SS,26,away,left,NaN,Richard


In [229]:
#splitting first and last name of player
df['player_last_name'] = df.displayName.str.split(' ', expand = True)[1]

In [230]:
#checking dataframe
df

,playDescription,quarter,down,yardsToGo,team_by_comp_yds,defendersInTheBox,numberOfPassRushers,QB_under_pressure,gameClock,absoluteYardlineNumber,...,nflId,displayName,jerseyNumber,position,frameId,team,playDirection,route,reciever_last_name,player_last_name
0,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,310.0,Matt Ryan,2.0,QB,36,away,left,NaN,Jones,Ryan
1,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,79848.0,Malcolm Jenkins,27.0,SS,36,home,left,NaN,Jones,Jenkins
2,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,2495454.0,Julio Jones,11.0,WR,36,away,left,HITCH,Jones,Jones
3,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,2495613.0,Corey Graham,24.0,FS,36,home,left,NaN,Jones,Graham
4,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,3,7.0,4.0,0.0,15:00:00,90.0,...,2533040.0,Mohamed Sanu,12.0,WR,36,away,left,HITCH,Jones,Sanu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13423,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,2556277.0,Marqui Christian,41.0,SS,26,away,left,NaN,Richard,Christian
13424,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,2556593.0,Cory Littleton,58.0,ILB,26,away,left,NaN,Richard,Littleton
13425,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,2556885.0,Jalen Richard,30.0,RB,26,home,left,SCREEN,Richard,Richard
13426,(:13) (Shotgun) D.Carr pass short right to J.R...,4,2,2,16,5.0,4.0,0.0,00:13:00,41.0,...,2558183.0,John Johnson,43.0,SS,26,away,left,NaN,Richard,Johnson


## Distance

In [231]:
play = df[df.playid == '201809060075']

In [236]:
play.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
playDescription,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...,(15:00) M.Ryan pass short right to J.Jones pus...
quarter,1,1,1,1,1,1,1,1,1,1,1,1,1,1
down,1,1,1,1,1,1,1,1,1,1,1,1,1,1
yardsToGo,15,15,15,15,15,15,15,15,15,15,15,15,15,15
team_by_comp_yds,3,3,3,3,3,3,3,3,3,3,3,3,3,3
defendersInTheBox,7,7,7,7,7,7,7,7,7,7,7,7,7,7
numberOfPassRushers,4,4,4,4,4,4,4,4,4,4,4,4,4,4
QB_under_pressure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gameClock,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00,15:00:00
absoluteYardlineNumber,90,90,90,90,90,90,90,90,90,90,90,90,90,90


In [237]:
#grabbing x coordinate from intended receiver
x1=play.loc[play.reciever_last_name == play.player_last_name].x.item()
x1

80.69

In [239]:
#grabbing y coordinate from intended receiver
y1=play.loc[play.reciever_last_name == play.player_last_name].y.item()
y1

44.91

In [276]:
shortest_distance = 100
shortest_x = 0
shortest_y = 0

#for x in coordinates of players
for x in play.x:
    #retrieve y coordinate
    y = play.loc[play.x == x].y.item()
    #if x does not equal original player
    if (x!=x1): 
        #solve for distance
        distance = ((x-x1)**2+(y-y1)**2)**(1/2)
        #if the distance is the shortest distance
        if distance < shortest_distance:
            #save the distance
            shortest_distance = distance
            #save the x coordinate
            shortest_x = x
            #save the y coordinate
            shortest_y = y
shortest_distance, shortest_x, shortest_y

(4.160048076645265, 76.53, 44.93)

In [240]:
#this is the correct answer for closest defender
x2=play.loc[play.displayName == 'Jalen Mills'].x.item()
y2=play.loc[play.displayName == 'Jalen Mills'].y.item()
x2,y2

(76.53, 44.93)

In [ ]:
#distance formula
((x2-x1)**2+(y2-y1)**2)**(1/2)

In [243]:
#getting all playids from week1
playids = [play for play in df.playid.unique()]

In [ ]:
#loop through each playid in playids
for play in playids:
      

# Drop Duplicate and Label Ball

In [ ]:
def prep_weeks(csv, number):
    week = pd.read_csv(csv)
    week.position = week.position.fillna('BALL')
    week = week.drop_duplicates()
    week = week.reset_index(drop = True)
    week.to_csv(f'week' + str(number) + '.csv')
    return week

In [ ]:
prep_weeks('week1.csv', 1)
print('week1')
prep_weeks('week2.csv', 2)
print('week2')
prep_weeks('week3.csv', 3)
print('week3')
prep_weeks('week4.csv', 4)
print('week4')
prep_weeks('week5.csv', 5)
print('week5')
prep_weeks('week6.csv', 6)
print('week6')
prep_weeks('week7.csv', 7)
print('week7')
prep_weeks('week8.csv', 8)
print('week8')
prep_weeks('week9.csv', 9)
print('week9')
prep_weeks('week10.csv', 10)
print('week10')
prep_weeks('week11.csv', 11)
print('week11')
prep_weeks('week12.csv', 12)
print('week12')
prep_weeks('week13.csv', 13)
print('week13')
prep_weeks('week14.csv', 14)
print('week14')
prep_weeks('week15.csv', 15)
print('week15')
prep_weeks('week16.csv', 16)
print('week16')
prep_weeks('week17.csv', 17)
print('week17')

# Exploration

## Library and Function for Field

In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly

In [ ]:
def create_animation(week_number,play_id):
    df = pd.read_csv('week'+ str(week_number) +'.csv')
    dd=df[(df.playId==play_id)]
    times = dd.time.unique()
    time_slice = times[0:500:5]
    df = dd[dd.time.isin(time_slice)]
                     
    colorsIdx = {'home': 'red', 'away': 'blue', 'football': 'orange'}
    fig = px.scatter(df, x='x', y='y', color='team', text='jerseyNumber', animation_frame='time', 
                     color_discrete_map=colorsIdx,
                     range_x=[-10, 110],range_y=[-10, 53.3],
                     hover_data=['displayName', 'position', 's', 'a', 'dis', 'o', 'playDirection']
                    )
    fig.update_traces(marker_size=20)
    
    fig.update_layout(paper_bgcolor='#29a500', plot_bgcolor='#29a500', font_color='white',
        width = 800,
        height = 600,
        title = "",
        
        xaxis = dict(
        nticks = 10,
        title = "",
        visible=False
        ),
        
        yaxis = dict(
        scaleanchor = "x",
        title = "Temp",
        visible=False
        ),
        showlegend= True,
  
        annotations=[
       dict(
            x=-5,
            y=12,
            xref="x",
            yref="y",
            text="HOME ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='bottom',
            textangle=-90
        ),
        dict(
            x=105,
            y=10,
            xref="x",
            yref="y",
            text="VISITOR ENDZONE",
            font=dict(size=16,color="#e9ece7"),
            align='center',
            showarrow=False,
            yanchor='bottom',
            textangle=90
        )]  
        ,
        legend=dict(
        traceorder="normal",
        font=dict(family="sans-serif",size=12),
        title = "",
        orientation="h",
        yanchor="bottom",
        y=1.00,
        xanchor="center",
        x=0.5
        ),
    )
    ####################################################
        
    fig.add_shape(type="rect", x0=-10, x1=0,  y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    fig.add_shape(type="rect", x0=100, x1=110, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#217b00" ,layer="below")
    for x in range(0, 100, 10):
        fig.add_shape(type="rect", x0=x,   x1=x+10, y0=0, y1=53.3,line=dict(color="#c8ddc0",width=3),fillcolor="#29a500" ,layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=1, x1=x, y1=2,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=51.3, x1=x, y1=52.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=20.0, x1=x, y1=21,line=dict(color="#c8ddc0",width=2),layer="below")
    for x in range(0, 100, 1):
        fig.add_shape(type="line",x0=x, y0=32.3, x1=x, y1=33.3,line=dict(color="#c8ddc0",width=2),layer="below")
    
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[5,5,5,5,5,5,5,5,5,5,5],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    fig.add_trace(go.Scatter(
    x=[2,10,20,30,40,50,60,70,80,90,98], y=[48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3,48.3],
    text=["G","1 0","2 0","3 0","4 0","5 0","4 0","3 0","2 0","1 0","G"],
    mode="text",
    textfont=dict(size=20,family="Arail"),
    showlegend=False,
    ))
    
    return fig

## Function test

In [ ]:
week1 = pd.read_csv('week1.csv')

In [ ]:
[play for play in week1.playId.unique()]

## Animation

In [ ]:
create_animation(1,75)

In [ ]:
week1 = pd.read_csv('week1.csv')

In [ ]:
week1

In [ ]:
week1.event.value_counts()

In [ ]:
df = week1[week1.event != 'None']

In [ ]:
df

# End